In [5]:
#pip uninstall -y opencv-python
#pip3 install opencv-contrib-python
#pip3 install imutils
import numpy as np 
import pandas as pd 
import cv2, os, re
import matplotlib.pyplot as plt
import imutils

from tqdm import tqdm
from imutils.object_detection import non_max_suppression

from tensorflow.keras.models import load_model

In [ ]:
# looking for the GPU on the device and how many they are
physical_devices = tf.config.list_physical_devices('GPU')
print(len(physical_devices))#if 0 then the computation is done on CPU
    

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/config/experimental/get_memory_growth
# Avoids OOM errors
if physical_devices is not None:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu,True)
else:
    print('No active GPU')

In [2]:
def find_regions(image, method):
    
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    
    if method == 'fast':
        ss.switchToSelectiveSearchFast()
    else:
        ss.switchToSelectiveSearchQuality()
    
    rects = ss.process()
    boxes = []
    for (x,y,w,h) in rects:
        
        boxes.append([x,y,w,h])
        pass
    
    return boxes

In [8]:
scene_path = 'C:/Users/tobia/Cumputor-vision/Data/scenes/scenes'

def load_scenes():
    
    scenes = []
    dirfiles = os.listdir(scene_path)
    dirfiles = sorted(dirfiles)
    for file in dirfiles:
        
        print(os.path.join(scene_path, file))
        scene = cv2.imread(os.path.join(scene_path, file))
        scene = cv2.cvtColor(scene, cv2.COLOR_BGR2RGB)
        scenes.append(scene)
        pass
    
    return scenes

In [9]:
scenes = load_scenes()

C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\lb_1.png
C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\lb_2.png
C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\lb_3.png
C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\lb_4.png
C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\sfbay_1.png
C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\sfbay_2.png
C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\sfbay_3.png
C:/Users/tobia/Cumputor-vision/Data/scenes/scenes\sfbay_4.png


In [12]:
%%time
method = "fast"

boxes = []

for scene in scenes:
    
    box_in_scene = find_regions(scene, method)
    boxes.append(box_in_scene)

AttributeError: module 'cv2' has no attribute 'ximgproc'

In [13]:
model = load_model('')

model.summary()

OSError: No file or directory found at 

In [14]:
rois, locs = [], []
images = []
for i,scene in tqdm(enumerate(scenes)):
    
    (H, W) = scene.shape[:2]
    region, loc = [],[]
    for (x,y,w,h) in boxes[i]:

        if w/float(W) > 0.10 and h/float(H) > 0.10:
            continue

        roi = scene[y:y+h,x:x+w]
        roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
        roi = cv2.resize(roi, (48,48))

        rois.append(roi)
        locs.append((x,y,x+w,y+h))
        pass
    
    preds = model.predict(np.array(rois,dtype=np.float32))
    preds = np.argmax(preds, axis=1)
    
    img = scene.copy()
    for (i,label) in enumerate(preds):

        if label == 1:
            (startX,startY,endX,endY) = locs[i]
            cv2.rectangle(img,(startX,startY),(endX,endY),(0,255,0),2)
        pass
    
    images.append(img)
    del rois[:]
    del locs[:]

0it [00:00, ?it/s]


IndexError: list index out of range